In [2]:
# Connect the colab notebook with drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics

In [4]:
dataset =  pd.read_csv("/content/drive/MyDrive/Customer_Churn/Cleaned_data_24_10_22.csv")
dataset.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df = dataset[['Geography', 'Gender','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','Exited']].copy()
df.head(5)

,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,0,0,42,2,0.00,1,1,1,1
1,2,0,41,1,83807.86,1,0,1,0
2,0,0,42,8,159660.80,3,1,0,1
3,0,0,39,1,0.00,2,0,0,0
4,2,0,43,2,125510.82,1,1,1,0


In [6]:
df.shape

(15682, 9)

In [7]:
X = dataset.drop(['Exited'],axis=1)
y = dataset['Exited']

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
mlp = MLPClassifier(max_iter=100)

parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

mlp_model = RandomizedSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
result = mlp_model.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [11]:
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.8345157221568136
Best Hyperparameters: {'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (100,), 'alpha': 0.05, 'activation': 'relu'}


In [12]:
# Predict values using the training data
mlp_cv_predict_train = mlp_model.predict(X_train)

print("Accuracy against training data: {0:.4f}".format(metrics.accuracy_score(y_train, mlp_cv_predict_train)))
print()

Accuracy against training data: 0.8483



In [13]:
# Predict values using the test data
mlp_cv_predict_test = mlp_model.predict(X_test)

print("Accuracy against test data: {0:.4f}".format(metrics.accuracy_score(y_test, mlp_cv_predict_test)))
print()

Accuracy against test data: 0.8346



In [14]:
print("Confusion Matrix")
print(metrics.confusion_matrix(y_test, mlp_cv_predict_test))
print()

Confusion Matrix
[[1291  270]
 [ 249 1327]]



In [15]:
print("Classification Report")
print(metrics.classification_report(y_test, mlp_cv_predict_test))
print()

Classification Report
              precision    recall  f1-score   support

           0       0.84      0.83      0.83      1561
           1       0.83      0.84      0.84      1576

    accuracy                           0.83      3137
   macro avg       0.83      0.83      0.83      3137
weighted avg       0.83      0.83      0.83      3137


